# Chat completions

In [1]:
base_url = "http://localhost:8080/v1"
api_key = "albert" # enter any value if no api_key is setup in config.ini file

## Without chat history

In [2]:
#!%pip install openai

In [3]:
# OpenAI client configuration
from openai import OpenAI

client = OpenAI(base_url=base_url, api_key=api_key)

In [4]:
# unstreamed chat
data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "Salut Albert !"}],
    "stream": False,
    "n": 1,
}

client.chat.completions.create(**data)

ChatCompletion(id='cmpl-380600d1542042ba81e3de8aee46f18d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Salut ! Comment vas-tu ?', role='assistant', function_call=None, tool_calls=[]), stop_reason=128009)], created=1719511595, model='AgentPublic/llama3-instruct-8b', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=14, total_tokens=23))

## With chat history

Because chat history need a `id` parameters (for chat ID) and this parameters is not currently supported by official OpenAI Python client, you need to request Albert API by *requests* npackage (native Python). We currently works to deploy a Albert Python client to support this parameters.   

In [5]:
# Request configuration
import requests

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}  # skip headers if no api_key is setup in config.ini file

The chat will keep the history of the conversation as long as the `user` parameter is set. 

In [6]:
# unstreamed chat
me = "leo"

data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content":"Salut Albert !"}],
    "stream": False,
    "n": 1,
    "user": me, # to start a conversation with chat history, setup a user name
}

response = session.post(url=f"{base_url}/chat/completions", json=data)

print(response.content)

b'{"id":"340bf5f9-3be0-4325-a1eb-44269d3dcb94","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"Salut ! Comment vas-tu aujourd\'hui ?","role":"assistant","function_call":null,"tool_calls":[]},"stop_reason":128009}],"created":1719511608,"model":"AgentPublic/llama3-instruct-8b","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":11,"prompt_tokens":14,"total_tokens":25}}'


Now, to continue the conversation with the current chat history, you need to pass the `id` parameter that was returned in the previous template response (here *340bf5f9-3be0-4325-a1eb-44269d3dcb94*).

In [7]:
chat_id = "340bf5f9-3be0-4325-a1eb-44269d3dcb94"

data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content":"Quel est mon précédent message ?"}],
    "stream": False,
    "n": 1,
    "user": me,
    "id": chat_id # to follow a conversation with chat history, setup the previous chat id
}
response = session.post(url=f"{base_url}/chat/completions", json=data)

print(response.content)

b'{"id":"340bf5f9-3be0-4325-a1eb-44269d3dcb94","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"Ton pr\xc3\xa9c\xc3\xa9dent message \xc3\xa9tait : \\"Salut Albert !\\"","role":"assistant","function_call":null,"tool_calls":[]},"stop_reason":128009}],"created":1719511628,"model":"AgentPublic/llama3-instruct-8b","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":13,"prompt_tokens":43,"total_tokens":56}}'


You can resume a past conversation by passing the `id` parameter and the correct chat ID. 